<a href="https://colab.research.google.com/github/jaysalomon/Bio-Spin/blob/main/Magnetosome_Array_Classifier_Simulation_(GPU_CuPy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cupy as cp # Import CuPy
from scipy.constants import mu_0, physical_constants
import matplotlib.pyplot as plt
import time
import warnings # To suppress CuPy experimental warnings if any

# Suppress potential warnings from CuPy's experimental features if used implicitly
warnings.filterwarnings("ignore", category=FutureWarning, module='cupy')

# --- GPU Device Check ---
try:
    cp.cuda.runtime.getDeviceCount()
    print("CUDA GPU detected and accessible via CuPy.")
except cp.cuda.runtime.CUDARuntimeError as e:
    print(f"CUDA Error: {e}")
    print("Please ensure CUDA toolkit is installed correctly and compatible with your CuPy version and NVIDIA driver.")
    exit()

# --- 1. Physical Parameters (Remain on CPU) ---
print("--- Defining Parameters ---")
gamma_e = physical_constants['electron gyromag. ratio'][0]
gamma_abs = np.abs(gamma_e)
gamma_LLG_H = gamma_abs * mu_0 # rad*m / (s*A)

Ms = 3e5  # A/m
diameter = 20e-9 # m
V = (4/3) * np.pi * (diameter/2)**3 # m^3
Ku = 1e4   # J/m^3
alpha = 0.05 # dimensionless

grid_size = 10
N = grid_size * grid_size
lattice_const = 40e-9 # m

T_sim = 2e-9   # s
dt = 1e-15     # s (Ensure this is small enough for stability with chosen solver)
num_steps = int(T_sim / dt)

f1 = 10e9  # Hz
f2 = 15e9  # Hz
H_amp = 1e4 # A/m

# --- 2. System Geometry & Coupling (Allocate on GPU) ---
print("--- Setting up Geometry on GPU ---")
# Create positions and easy axes on CPU first, then transfer
particle_positions_np = np.zeros((N, 3))
easy_axes_np = np.zeros((N, 3))
idx = 0
for i in range(grid_size):
    for j in range(grid_size):
        particle_positions_np[idx] = np.array([i * lattice_const, j * lattice_const, 0])
        rand_vec = np.random.randn(3)
        easy_axes_np[idx] = rand_vec / np.linalg.norm(rand_vec)
        idx += 1

# Transfer to GPU
particle_positions = cp.asarray(particle_positions_np)
easy_axes = cp.asarray(easy_axes_np)

# --- 3. LLG Equation Core (Using CuPy functions) ---
print("--- Defining Physics Functions (GPU operations) ---")

def calculate_effective_field_gpu(m_array, H_ext_t, easy_axes, particle_positions, N, Ms, V, Ku):
    """Calculates the effective magnetic field H_eff (A/m) for each particle using CuPy."""
    # Ensure H_ext_t is a CuPy array on the GPU
    H_ext_t_gpu = cp.asarray(H_ext_t) # Transfers if H_ext_t is numpy/list

    # Initialize H_eff on GPU
    H_eff_array = cp.zeros_like(m_array)

    # 1. External Field (Uniform for all particles)
    # Use broadcasting: H_ext_t_gpu is (3,), reshape to (1, 3) to broadcast over N particles
    H_eff_array += H_ext_t_gpu.reshape(1, 3)

    # 2. Anisotropy Field
    # H_k = (2 * Ku / (mu_0 * Ms)) * (m . easy_axis) * easy_axis
    m_dot_easy = cp.sum(m_array * easy_axes, axis=1) # Dot product for each particle (N,)
    # Reshape m_dot_easy to (N, 1) for broadcasting with easy_axes (N, 3)
    H_anisotropy = (2 * Ku / (mu_0 * Ms)) * (m_dot_easy[:, cp.newaxis] * easy_axes)
    H_eff_array += H_anisotropy

    # 3. Dipolar Coupling Field (Vectorized GPU implementation)
    # Calculate pairwise vectors r_ij = pos_i - pos_j
    # positions shape: (N, 3)
    # Expand dims for broadcasting: pos_i -> (N, 1, 3), pos_j -> (1, N, 3)
    r_vecs = particle_positions[:, None, :] - particle_positions[None, :, :] # Shape (N, N, 3)

    # Calculate pairwise distances r_ij_mag
    r_mags_sq = cp.sum(r_vecs**2, axis=2) # Shape (N, N), includes zeros on diagonal
    # Add small epsilon for stability where r_mags_sq is zero (i=j)
    # Or handle division by zero later. Let's use epsilon.
    epsilon = 1e-30
    r_mags = cp.sqrt(r_mags_sq + epsilon) # Shape (N, N)

    # Calculate unit vectors r_ij_hat = r_ij_vec / r_ij_mag
    # Need to handle division by zero for i=j. r_mags has epsilon there.
    r_hat = r_vecs / (r_mags[:, :, None] + epsilon) # Shape (N, N, 3)

    # Calculate m_j . r_ij_hat term
    # m_array shape: (N, 3) -> expand for broadcasting: m_j -> (1, N, 3)
    m_j = m_array[None, :, :]
    m_dot_rhat = cp.sum(m_j * r_hat, axis=2) # Shape (N, N)

    # Calculate 3 * (m_j . r_ij_hat) * r_ij_hat - m_j
    term1 = 3 * m_dot_rhat[:, :, None] * r_hat # Shape (N, N, 3)
    term2 = m_j # Shape (1, N, 3)

    # Calculate H_dip_tensor (field from j on i)
    # Need r_mags**3, handle diagonal carefully
    r_mags_cubed = r_mags**3
    # Avoid division by zero on diagonal by adding epsilon or setting diagonal later
    H_dip_tensor = (Ms * V / (4 * cp.pi * (r_mags_cubed[:, :, None] + epsilon))) * (term1 - term2) # Shape (N, N, 3)

    # Zero out self-interaction terms (diagonal i=j)
    # Create an identity matrix, expand dims, and multiply to zero out diagonal
    identity = cp.eye(N, dtype=bool) # Shape (N, N)
    H_dip_tensor[identity] = 0.0 # Set diagonal elements (where i==j) to zero

    # Sum over j to get total dipolar field on i
    H_dipolar = cp.sum(H_dip_tensor, axis=1) # Shape (N, 3)

    H_eff_array += H_dipolar

    return H_eff_array

def llg_rhs_gpu(m, H_eff, alpha, gamma_LLG_H):
    """Calculates the right-hand side of the LLG equation (dm/dt) using CuPy."""
    prefactor1 = -gamma_LLG_H / (1 + alpha**2)
    prefactor2 = alpha * prefactor1

    # Use cp.cross for cross products
    m_cross_Heff = cp.cross(m, H_eff)
    m_cross_m_cross_Heff = cp.cross(m, m_cross_Heff)

    dm_dt = prefactor1 * m_cross_Heff + prefactor2 * m_cross_m_cross_Heff
    return dm_dt

def calculate_damping_power_gpu(m, H_eff, alpha, gamma_LLG_H, Ms, V):
    """Calculates instantaneous power dissipated due to damping using CuPy."""
    m_cross_Heff = cp.cross(m, H_eff)
    # Calculate magnitude squared: sum of squares along the vector axis (axis=1 for (N,3) array)
    m_cross_Heff_mag_sq = cp.sum(m_cross_Heff**2, axis=1) # Shape (N,)
    power = (alpha * gamma_LLG_H * V * Ms) / (1 + alpha**2) * m_cross_Heff_mag_sq
    # Return total power by summing over all particles
    return cp.sum(power)


# --- 4. Input Signals (Return NumPy arrays, will be transferred) ---
print("--- Defining Input Signals ---")
def input_signal_red(t):
    """Input signal 1: Sinusoidal field. Returns NumPy array."""
    return np.array([0, 0, H_amp * np.sin(2 * np.pi * f1 * t)])

def input_signal_green(t):
    """Input signal 2: Different frequency sinusoidal field. Returns NumPy array."""
    return np.array([0, 0, H_amp * np.sin(2 * np.pi * f2 * t)])

# --- 5. Simulation Function (Operates on GPU) ---
print("--- Defining Simulation Function (GPU execution) ---")
def run_simulation_gpu(input_signal_func, T_sim, dt, m_initial_gpu, easy_axes_gpu, particle_positions_gpu, N, Ms, V, Ku, alpha, gamma_LLG_H):
    """Runs the LLG simulation on the GPU using CuPy."""
    print(f"Starting GPU simulation for {input_signal_func.__name__}...")
    start_time = time.time()

    # Ensure initial state is on GPU
    m_current = m_initial_gpu.copy()

    # History lists will store CPU data (NumPy arrays)
    m_history_cpu = []
    power_history_cpu = []
    total_energy_dissipated = 0.0 # Accumulate on CPU

    num_steps = int(T_sim / dt)
    # Use CUDA events for more accurate GPU timing (optional)
    # start_event = cp.cuda.Event()
    # stop_event = cp.cuda.Event()
    # start_event.record()

    for step in range(num_steps):
        t = step * dt

        # Get external field (NumPy array), transfer handled in calculate_effective_field_gpu
        H_ext_t = input_signal_func(t)

        # Calculate effective field for all particles on GPU
        H_eff = calculate_effective_field_gpu(m_current, H_ext_t, easy_axes_gpu, particle_positions_gpu, N, Ms, V, Ku)

        # Calculate dm/dt for all particles on GPU (vectorized)
        dm_dt = llg_rhs_gpu(m_current, H_eff, alpha, gamma_LLG_H)

        # Calculate total instantaneous power on GPU
        instantaneous_total_power_gpu = calculate_damping_power_gpu(m_current, H_eff, alpha, gamma_LLG_H, Ms, V)

        # Update m using Euler method on GPU
        m_next = m_current + dm_dt * dt

        # Renormalize magnetization vectors on GPU
        norm = cp.linalg.norm(m_next, axis=1)
        # Add epsilon to norm to prevent division by zero if norm is zero
        epsilon_norm = 1e-12
        m_next = m_next / (norm[:, cp.newaxis] + epsilon_norm)

        m_current = m_next

        # Transfer power value to CPU for accumulation and history
        instantaneous_total_power_cpu = cp.asnumpy(instantaneous_total_power_gpu)
        total_energy_dissipated += instantaneous_total_power_cpu * dt

        # Store results in CPU lists (transfer data back)
        if step % 1000 == 0: # Store less frequently
             m_history_cpu.append(cp.asnumpy(m_current.copy())) # Transfer m state to CPU
             power_history_cpu.append(instantaneous_total_power_cpu)
             # print(f"Step {step}/{num_steps}, t = {t*1e12:.2f} ps") # Progress indicator

        # Simple stability check on GPU
        if cp.any(cp.isnan(m_current)):
            print(f"Warning: Simulation became unstable at step {step}!")
            # Transfer final state back even if unstable
            m_final_cpu = cp.asnumpy(m_current)
            return m_final_cpu, total_energy_dissipated, np.array(m_history_cpu), np.array(power_history_cpu)


    # stop_event.record()
    # stop_event.synchronize()
    # gpu_time = cp.cuda.get_elapsed_time(start_event, stop_event) / 1000.0 # Time in seconds
    # print(f"GPU computation finished in {gpu_time:.2f} seconds (measured by CUDA events).")

    end_time = time.time()
    sim_duration = end_time - start_time
    print(f"Total simulation function finished in {sim_duration:.2f} seconds (includes data transfers).")

    # Transfer final state back to CPU
    m_final_cpu = cp.asnumpy(m_current)

    return m_final_cpu, total_energy_dissipated, np.array(m_history_cpu), np.array(power_history_cpu)

# --- 6. Classification & Analysis (Operate on CPU data) ---
print("--- Running Simulations on GPU ---")

# Initial state on GPU
# m_initial_np = np.random.randn(N, 3)
# m_initial_np /= np.linalg.norm(m_initial_np, axis=1)[:, np.newaxis]
# m_initial = cp.asarray(m_initial_np)

# Initial state: All aligned along +X (example)
m_initial_np = np.zeros((N, 3))
m_initial_np[:, 0] = 1.0
m_initial = cp.asarray(m_initial_np) # Transfer to GPU

# Run Simulation 1: Red Light (GPU)
# Pass GPU arrays to the simulation function
m_final_red_cpu, energy_red, m_hist_red_cpu, p_hist_red_cpu = run_simulation_gpu(
    input_signal_red, T_sim, dt, m_initial, easy_axes, particle_positions, N, Ms, V, Ku, alpha, gamma_LLG_H
)

# Run Simulation 2: Green Light (GPU) - Reset initial state on GPU
# Re-transfer or recreate the initial state on GPU if needed
m_initial = cp.asarray(m_initial_np) # Ensure clean start state on GPU
m_final_green_cpu, energy_green, m_hist_green_cpu, p_hist_green_cpu = run_simulation_gpu(
    input_signal_green, T_sim, dt, m_initial, easy_axes, particle_positions, N, Ms, V, Ku, alpha, gamma_LLG_H
)

print("\n--- Classification Check (using CPU data) ---")
# Results are already on CPU (NumPy arrays)
avg_m_final_red = np.mean(m_final_red_cpu, axis=0)
avg_m_final_green = np.mean(m_final_green_cpu, axis=0)

print(f"Average final magnetization (Red Signal):   {avg_m_final_red}")
print(f"Average final magnetization (Green Signal): {avg_m_final_green}")

state_distance = np.linalg.norm(avg_m_final_red - avg_m_final_green)
print(f"Distance between average final states: {state_distance:.4f}")

if state_distance > 0.1:
    print("The final states are significantly distinct. Classification is possible.")
else:
    print("The final states are similar. Classification may be difficult.")

# --- 7. Parameter Estimation Outputs ---
print("\n--- Performance Estimation ---")
# Energy values are already calculated on CPU
print(f"Estimated Intrinsic Energy Dissipated (Red Signal):   {energy_red:.4e} Joules")
print(f"Estimated Intrinsic Energy Dissipated (Green Signal): {energy_green:.4e} Joules")
print("Note: Energy calculated based on GPU power values transferred to CPU.")

T_task = T_sim
print(f"\nEstimated Task Latency (T_task): {T_task:.2e} seconds")
print(f"Potential Throughput: ~ 1 / (T_task + T_reset) ops/sec.")
print(f"      For this simulation T_task = T_sim = {T_sim * 1e9:.1f} ns")

# --- 8. Visualization (Using CPU data) ---
print("\n--- Plotting Results ---")
# All plotting data should be NumPy arrays on the CPU
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Check if history arrays are empty before plotting
if m_hist_red_cpu.size > 0:
    time_axis_history = np.linspace(0, T_sim, len(m_hist_red_cpu)) * 1e9 # Time in ns
    avg_m_hist_red = np.mean(m_hist_red_cpu, axis=1)
    axes[0, 0].plot(time_axis_history, avg_m_hist_red[:, 0], label='Mx_avg')
    axes[0, 0].plot(time_axis_history, avg_m_hist_red[:, 1], label='My_avg')
    axes[0, 0].plot(time_axis_history, avg_m_hist_red[:, 2], label='Mz_avg')
axes[0, 0].set_title('Avg. Magnetization vs Time (Red Signal)')
axes[0, 0].set_xlabel('Time (ns)')
axes[0, 0].set_ylabel('Avg. Magnetization Component')
axes[0, 0].legend()
axes[0, 0].grid(True)

if m_hist_green_cpu.size > 0:
    time_axis_history = np.linspace(0, T_sim, len(m_hist_green_cpu)) * 1e9 # Time in ns
    avg_m_hist_green = np.mean(m_hist_green_cpu, axis=1)
    axes[0, 1].plot(time_axis_history, avg_m_hist_green[:, 0], label='Mx_avg')
    axes[0, 1].plot(time_axis_history, avg_m_hist_green[:, 1], label='My_avg')
    axes[0, 1].plot(time_axis_history, avg_m_hist_green[:, 2], label='Mz_avg')
axes[0, 1].set_title('Avg. Magnetization vs Time (Green Signal)')
axes[0, 1].set_xlabel('Time (ns)')
axes[0, 1].set_ylabel('Avg. Magnetization Component')
axes[0, 1].legend()
axes[0, 1].grid(True)

if p_hist_red_cpu.size > 0 and p_hist_green_cpu.size > 0:
     time_axis_power = np.linspace(0, T_sim, len(p_hist_red_cpu)) * 1e9 # Time in ns
     axes[1, 0].plot(time_axis_power, p_hist_red_cpu * 1e12, label='Red Signal') # Power in pW
     axes[1, 0].plot(time_axis_power, p_hist_green_cpu * 1e12, label='Green Signal') # Power in pW
     axes[1, 0].set_title('Total Instantaneous Power Dissipation vs Time')
     axes[1, 0].set_xlabel('Time (ns)')
     axes[1, 0].set_ylabel('Power (pW)')
     axes[1, 0].legend()
     axes[1, 0].grid(True)
     axes[1, 0].set_yscale('log')

# Use initial positions (NumPy version) and final states (CPU versions) for quiver plot
axes[1, 1].quiver(particle_positions_np[:, 0]*1e9, particle_positions_np[:, 1]*1e9,
                   m_final_red_cpu[:, 0], m_final_red_cpu[:, 1], color='red', scale=grid_size, label='Red Final State')
axes[1, 1].quiver(particle_positions_np[:, 0]*1e9, particle_positions_np[:, 1]*1e9,
                   m_final_green_cpu[:, 0], m_final_green_cpu[:, 1], color='green', scale=grid_size, label='Green Final State', alpha=0.7)
axes[1, 1].set_title('Final Magnetization States (XY Projection)')
axes[1, 1].set_xlabel('X position (nm)')
axes[1, 1].set_ylabel('Y position (nm)')
axes[1, 1].set_aspect('equal', adjustable='box')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print("\n--- GPU Simulation Complete ---")